## Goal here is to pre-process my abstracts text for topic modeling ##

Some notes
----
document to topic weights

\# of documents in topic

df of top documents within a given topic

list of top tokens within topic

Click document -> highlight topic tokens

Set high k for lda

In [2]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import urllib
import time
import feedparser
%matplotlib inline

from gensim import corpora, models, similarities
from gensim.matutils import Sparse2Corpus
from gensim.parsing.preprocessing import STOPWORDS
from time import time

import pickle


In [6]:
# Gonna just import the whole batch for now, b/c it's small enough and streaming isn't necessary
abstracts = pd.read_csv('./new_hope_data/arxiv_csML.csv')['summary']

In [12]:
stopfree= [[word for word in doc.lower().split(' ') if word not in STOPWORDS ] for doc in abstracts[:5]]

stopfree

[['discuss',
  'algorithms',
  'estimating',
  'shannon',
  'entropy',
  'h',
  'finite',
  'symbol',
  'sequences',
  'long',
  'range',
  'correlations.',
  'particular,',
  'consider',
  'algorithms',
  'estimate',
  'h',
  'code',
  'lengths',
  'produced',
  'compression',
  'algorithm.',
  'describing',
  'convergence',
  'sequence',
  'length,',
  'assuming',
  'limits',
  'space',
  'time',
  'complexities',
  'compression',
  'algorithms.',
  'scaling',
  'law',
  'proposed',
  'extrapolation',
  'finite',
  'sample',
  'lengths.',
  'applied',
  'sequences',
  'dynamical',
  'systems',
  'non-trivial',
  'chaotic',
  'regimes,',
  '1-d',
  'cellular',
  'automaton,',
  'written',
  'english',
  'texts.'],
 ['designing',
  'photometric',
  'best',
  'fulfil',
  'set',
  'scientific',
  'goals',
  'complex',
  'task,',
  'demanding',
  'compromise',
  'conflicting',
  'requirements',
  'subject',
  'constraints.',
  'specific',
  'example',
  'determination',
  'stellar',
  'as

In [16]:
# from gensim.corpora.textcorpus.TextCorpus import get_texts
# import gensim.copora.textcorpus as tc
import gensim.corpora.textcorpus as tc

In [28]:
import spacy
# spacy.load('en')
from spacy.lang.en import English
parser = English()

In [30]:
test = abstracts[0]
parsed_data = parser(test)

In [43]:
import nltk
nltk.download('wordnet')

from nltk.corpus import wordnet as wn

[nltk_data] Downloading package wordnet to /home/omar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [44]:
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [45]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/omar/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [128]:
def prep_text(text):
    myreg=r'\$.+\$|\[.+\]|\(.+\, *\d{2,4}\w*\)' #this removes LATEX formatting as well as (most) citations
    parsed_data = parser(re.sub(myreg, '', text))
    tokens = [str(token).lower() for token in parsed_data if len(token) > 3]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [129]:
monster_toke = [prep_text(abstract) for abstract in abstracts] # Oh god I hope my comp doesn't freak

In [130]:
dictionary = corpora.Dictionary(monster_toke)

In [131]:
dictionary.save("./new_hope_data/monster_toke.dict")

In [132]:
print(dictionary.token2id)

{'algorithm': 0, 'apply': 1, 'assume': 2, 'automaton': 3, 'cellular': 4, 'chaotic': 5, 'code': 6, 'complexity': 7, 'compression': 8, 'consider': 9, 'convergence': 10, 'correlation': 11, 'describe': 12, 'discus': 13, 'dynamical': 14, 'english': 15, 'entropy': 16, 'estimate': 17, 'extrapolation': 18, 'finite': 19, 'interest': 20, 'length': 21, 'limit': 22, 'long': 23, 'particular': 24, 'produce': 25, 'propose': 26, 'range': 27, 'regime': 28, 'sample': 29, 'scaling': 30, 'sequence': 31, 'shannon': 32, 'space': 33, 'symbol': 34, 'system': 35, 'text': 36, 'time': 37, 'trivial': 38, 'write': 39, 'across': 40, 'although': 41, 'approach': 42, 'assumption': 43, 'astrometry': 44, 'astrophysical': 45, 'avoid': 46, 'best': 47, 'biology': 48, 'broad': 49, 'carry': 50, 'complex': 51, 'compromise': 52, 'conflict': 53, 'constraint': 54, 'degeneracy': 55, 'demand': 56, 'design': 57, 'designing': 58, 'determination': 59, 'different': 60, 'direction': 61, 'effective': 62, 'evolutionary': 63, 'example': 6

In [135]:
corpus = [dictionary.doc2bow(text) for text in monster_toke]
corpora.MmCorpus.serialize('./new_hope_data/monster_toke_corp.mm', corpus)

## Ok that's good for now. We have our dictionary and corpus. It's not perfect yet, but eh..meh. we'll see ##

In [2]:
# Load the data from it's saved form
dictionary = corpora.Dictionary.load('./new_hope_data/monster_toke.dict')
corpus = corpora.MmCorpus('./new_hope_data/monster_toke_corp.mm')

In [ ]:
# Initialize a model, TFIDF
tfidf = models.TfidfModel(corpus) # step 1 -- initialize a model

In [7]:
tfidf_corpus = tfidf[corpus]

In [11]:
# Initialize LSI based on TFIDF
lsi = models.LsiModel(tfidf_corpus, id2word=dictionary, num_topics=4) # 10 for now
corpus_lsi = lsi[tfidf_corpus]

In [12]:
lsi.print_topics(4)
# These topics are horrible!

[(0,
  '0.138*"model" + 0.129*"network" + 0.123*"algorithm" + 0.117*"data" + 0.117*"method" + 0.114*"learning" + 0.102*"problem" + 0.096*"function" + 0.092*"graph" + 0.091*"feature"'),
 (1,
  '-0.268*"network" + 0.213*"matrix" + -0.193*"deep" + -0.187*"neural" + 0.158*"convex" + -0.141*"training" + 0.138*"kernel" + -0.134*"image" + -0.133*"adversarial" + -0.125*"task"'),
 (2,
  '0.499*"graph" + 0.246*"clustering" + -0.240*"gradient" + -0.169*"policy" + 0.151*"kernel" + -0.141*"optimization" + 0.139*"cluster" + -0.138*"stochastic" + -0.129*"convex" + 0.122*"node"'),
 (3,
  '0.271*"inference" + -0.258*"graph" + 0.223*"variational" + 0.212*"bayesian" + -0.206*"matrix" + 0.193*"distribution" + 0.185*"variable" + 0.185*"posterior" + 0.181*"model" + 0.175*"latent"')]

In [15]:
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=2)
corpus_lda = lda[corpus]
lda.print_topics(2)

[(0,
  '0.018*"model" + 0.015*"learning" + 0.013*"network" + 0.011*"data" + 0.009*"method" + 0.008*"propose" + 0.007*"neural" + 0.007*"approach" + 0.007*"deep" + 0.007*"base"'),
 (1,
  '0.015*"algorithm" + 0.014*"method" + 0.012*"problem" + 0.011*"data" + 0.009*"propose" + 0.007*"show" + 0.007*"model" + 0.007*"function" + 0.006*"result" + 0.006*"learning"')]

In [14]:
hdp = models.HdpModel(corpus, id2word=dictionary)
corpus_hdp= hdp[corpus]
hdp.print_topics()

[(0,
  '0.015*model + 0.012*data + 0.011*method + 0.010*learning + 0.010*algorithm + 0.008*propose + 0.008*problem + 0.007*network + 0.006*show + 0.006*base'),
 (1,
  '0.015*model + 0.012*data + 0.012*learning + 0.010*method + 0.009*network + 0.008*algorithm + 0.008*propose + 0.007*problem + 0.006*show + 0.006*base'),
 (2,
  '0.009*model + 0.008*algorithm + 0.007*method + 0.007*data + 0.006*learning + 0.005*propose + 0.005*problem + 0.005*network + 0.005*base + 0.004*show'),
 (3,
  '0.009*model + 0.006*data + 0.005*method + 0.005*algorithm + 0.004*problem + 0.004*learning + 0.004*network + 0.004*propose + 0.003*show + 0.003*result'),
 (4,
  '0.004*model + 0.003*algorithm + 0.003*method + 0.003*data + 0.002*result + 0.002*base + 0.002*approach + 0.002*problem + 0.002*show + 0.002*propose'),
 (5,
  '0.003*model + 0.003*algorithm + 0.002*data + 0.002*learning + 0.001*result + 0.001*show + 0.001*problem + 0.001*base + 0.001*matrix + 0.001*approach'),
 (6,
  '0.002*method + 0.002*network + 

In [3]:
# Going to follow along with https://github.com/bhargavvader/personal/tree/master/notebooks/text_analysis_tutorial
import matplotlib.pyplot as plt
import gensim
import numpy as np
import spacy

from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary
import pyLDAvis.gensim

import os, re, operator, warnings

In [10]:
# Gonna just import the whole batch for now, b/c it's small enough and streaming isn't necessary
abstracts = pd.read_csv('./new_hope_data/arxiv_csML.csv')['summary']

In [15]:
import spacy
nlp = spacy.load('en') #en

In [17]:
preproc_abstracts = [[w.lemma_ for w in nlp(abstract) if not w.is_stop and not w.is_punct and not w.like_num] for abstract in abstracts]
# for abstract in abstracts:
#     curr_abst = nlp(abstract)
#     new_abst = [[w.lemma_ for w in nlp(abstract) if not w.is_stop and not w.is_punct and not w.like_num] for abstract in abstracts]

In [19]:
with open('./new_hope_data/preproc_abstracts', 'wb') as fp:
    pickle.dump(preproc_abstracts, fp)


In [5]:
with open ('./new_hope_data/preproc_abstracts', 'rb') as fp:
    corpus = pickle.load(fp)

In [7]:
bigrams = gensim.models.Phrases(corpus)

In [8]:
texts = [bigrams[line] for line in corpus]

/home/omar/miniconda3/lib/python3.6/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [9]:
with open('./new_hope_data/abstract_bigrams', 'wb') as fp:
    pickle.dump(texts, fp)

In [10]:
## USE THIS ONE
with open ('./new_hope_data/abstract_bigrams', 'rb') as fp:
    texts = pickle.load(fp)

In [10]:
dictionary = Dictionary(texts)

In [11]:
# Save the dict
dictionary.save('./new_hope_data/new_try.dict')

In [4]:
dct = Dictionary.load('./new_hope_data/new_try.dict')
len(dct)

43711

In [8]:
len(dictionary)

43711

In [14]:
# before filtering, length was 43711 Now filtering
dct.filter_extremes(no_below = 5, no_above = .9)
len(dct)

13580

In [34]:
print(dct.token2id)

{'1-d': 0, 'a': 1, 'algorithm': 2, 'apply': 3, 'assume': 4, 'automaton': 5, 'cellular': 6, 'chaotic': 7, 'code': 8, 'complexity': 9, 'compression': 10, 'consider': 11, 'convergence': 12, 'correlation': 13, 'describe': 14, 'discuss': 15, 'dynamical_system': 16, 'english': 17, 'estimate': 18, 'extrapolation': 19, 'finite': 20, 'finite_sample': 21, 'h': 22, 'in_particular': 23, 'interest': 24, 'law': 25, 'length': 26, 'limit': 27, 'long_range': 28, 'non_trivial': 29, 'produce': 30, 'regime': 31, 'scale': 32, 'sequence': 33, 'shannon_entropy': 34, 'space': 35, 'symbol_sequence': 36, 'text': 37, 'this': 38, 'time': 39, 'write': 40, '`': 41, 'ap': 42, 'approach': 43, 'aps': 44, 'astrometry': 45, 'astrophysical': 46, 'avoid': 47, 'best': 48, 'biology': 49, 'broad': 50, 'by': 51, 'carry': 52, 'complex': 53, 'compromise': 54, 'conflicting': 55, 'degeneracy': 56, 'demand': 57, 'design': 58, 'determination': 59, 'different': 60, 'direction': 61, 'effective': 62, 'etc': 63, 'evolutionary': 64, 'ex

In [15]:
corpus = [dct.doc2bow(text) for text in texts]

In [23]:
lsimodel = LsiModel(corpus=corpus, num_topics=10, id2word=dct)
lsimodel.show_topics(num_topics=5)  # Showing only the top 5 topics

[(0,
  '0.458*"-PRON-" + 0.381*"model" + 0.258*"method" + 0.235*"algorithm" + 0.211*"datum" + 0.172*"the" + 0.168*"problem" + 0.162*"propose" + 0.149*"base" + 0.138*"approach"'),
 (1,
  '0.799*"model" + -0.342*"algorithm" + -0.293*"$" + -0.187*"problem" + -0.177*"-PRON-" + -0.133*"method" + -0.065*"function" + -0.059*"matrix" + 0.054*"datum" + -0.045*"result"'),
 (2,
  '-0.595*"$" + 0.505*"method" + -0.330*"model" + -0.249*"algorithm" + 0.224*"datum" + 0.162*"network" + -0.127*"-PRON-" + 0.109*"propose" + 0.095*"approach" + 0.089*"feature"'),
 (3,
  '-0.570*"$" + -0.537*"method" + 0.449*"algorithm" + 0.253*"-PRON-" + 0.177*"network" + 0.151*"learn" + -0.089*"model" + 0.071*"task" + -0.064*"the" + 0.063*"learning"'),
 (4,
  '0.557*"algorithm" + -0.498*"network" + -0.462*"-PRON-" + -0.184*"$" + 0.164*"model" + 0.150*"method" + 0.147*"the" + 0.140*"datum" + 0.114*"propose" + 0.083*"problem"')]

In [12]:
suspects= ["-PRON-", "model", "method", "propose"]
for susp in suspects:
    counter = 0
    for doc in texts:
        if susp in doc:
            counter +=1
    print(f"{susp} found in {counter/len(texts)}% of docs")

-PRON- found in 0.801592742998787% of docs
model found in 0.49886609356046624% of docs
method found in 0.4697009651389695% of docs
propose found in 0.42561046358314436% of docs


In [13]:
"model" in dct.token2id

True

In [14]:
# So the suspects didn't meet the filtering criteria, so now I have to go in and manually delete this crap
dct.filter_tokens(bad_ids=[dct.token2id[w] for w in suspects])

In [15]:
"model" in dct.token2id

False

In [16]:
corpus = [dct.doc2bow(text) for text in texts]

In [17]:
# OK let's try again w/ topic modeling: 
lsimodel = LsiModel(corpus=corpus, num_topics=10, id2word=dct)
lsimodel.show_topics(num_topics=5)  # Showing only the top 5 topics

[(0,
  '0.344*"algorithm" + 0.288*"datum" + 0.238*"the" + 0.232*"problem" + 0.197*"base" + 0.181*"approach" + 0.170*"network" + 0.163*"learn" + 0.159*"$" + 0.136*"result"'),
 (1,
  '-0.698*"$" + -0.381*"algorithm" + 0.277*"datum" + 0.275*"network" + -0.156*"problem" + 0.128*"approach" + 0.121*"learn" + 0.099*"task" + 0.091*"feature" + -0.080*"matrix"'),
 (2,
  '-0.682*"algorithm" + 0.600*"$" + 0.307*"network" + -0.098*"problem" + 0.083*"datum" + 0.048*"feature" + 0.047*"training" + 0.047*"train" + 0.042*"neural_network" + 0.042*"distribution"'),
 (3,
  '0.711*"network" + -0.548*"datum" + 0.235*"algorithm" + 0.132*"learn" + -0.095*"approach" + 0.084*"node" + -0.075*"the" + 0.072*"graph" + 0.068*"task" + -0.066*"kernel"'),
 (4,
  '0.590*"datum" + -0.421*"problem" + 0.374*"algorithm" + 0.285*"network" + -0.215*"approach" + 0.169*"$" + -0.137*"function" + -0.133*"base" + -0.078*"kernel" + -0.074*"task"')]

In [18]:
parser = spacy.load('en')

In [23]:
omg = parser('the')

In [27]:
for i in omg:
    print(i.is_stop)

True


In [33]:
lsimodel.show_topic(2)

[('algorithm', -0.6823068138399734),
 ('$', 0.5996840099246906),
 ('network', 0.30678796048102386),
 ('problem', -0.09843733540989567),
 ('datum', 0.08252593487416394),
 ('feature', 0.047687905846779545),
 ('training', 0.04679997206122962),
 ('train', 0.046614920942173795),
 ('neural_network', 0.04190823941331686),
 ('distribution', 0.041569007826928225)]